In [4]:
import pygimli as pg
import numpy as np
import pandas as pd
from scipy.spatial import Delaunay

# Example dataset with x, z, and conductivity
data = pd.DataFrame({
    'x': [0, 1, 2, 0, 1, 2],
    'z': [0, 0, 0, -1, -1, -1],
    'conductivity': [10, 15, 20, 12, 18, 22]
})

# Extract x, z, and conductivity
x = data['x'].values
z = data['z'].values
conductivity = data['conductivity'].values

# Combine x and z into a 2D NumPy array for Delaunay triangulation
points = np.column_stack((x, z))  # Ensures points is a 2D NumPy array

# Perform Delaunay triangulation
tri = Delaunay(points)

# Create an unstructured PyGIMLi mesh
mesh = pg.Mesh(2)

# Add nodes to the mesh
node_map = {}
for i, point in enumerate(points):
    node = mesh.createNode(point)
    node_map[i] = node

# Add triangles to the mesh
for triangle in tri.simplices:
    mesh.createTriangle(
        node_map[triangle[0]],
        node_map[triangle[1]],
        node_map[triangle[2]]
    )

# Assign conductivity values to cells
cell_values = []
for cell in mesh.cells():
    # Calculate the mean conductivity for the cell's nodes
    cell_nodes = cell.nodes()
    node_indices = [list(points).index(node.pos()) for node in cell_nodes]
    cell_conductivity = np.mean([conductivity[i] for i in node_indices])
    cell_values.append(cell_conductivity)

mesh['conductivity'] = cell_values

# Visualize the mesh with conductivity
pg.show(mesh, data=mesh['conductivity'], label='Conductivity', cMap='viridis')


ValueError: operands could not be broadcast together with shapes (2,) (3,) 